In [1]:
import pandas as pd

In [2]:
import numpy as np

On commence par charger les données qui sont dans un fichier .csv avec comme séparateur une ,

In [3]:
file_name="dispersion_10240.csv"

In [4]:
xy_train=pd.read_csv(file_name,header=0)

Visualisation

Pour voir le format des données

In [29]:
xy_train.shape

(10240, 17)

In [82]:
xy_train['model'] = xy_train['model'].astype('category')

In [83]:
xy_train['ihaze'] = xy_train['ihaze'].astype('category')

In [84]:
xy_train['clouds'] = xy_train['clouds'].astype('category')

In [85]:
x_tot=np.array(xy_train[['cap','Mach','pitch','E_e_ai','power_setting','altitude','roll','yaw','hour','model','ihaze','hbase','hr','ta','vis','clouds']])

In [86]:
y_tot=np.array(xy_train['IRS'])

In [91]:
x_tot.shape

(10240, 16)

**Modèle de type régression linéaire**

In [13]:
import sklearn.linear_model as lm
linreg = lm.LinearRegression()

In [35]:
y_tot.shape

(10240,)

Il faut un vecteur de type (10240,1) pour l'apprentissage, on fait un reshape

In [36]:
y_tot=y_tot.reshape(10240,1)

In [37]:
lr=linreg.fit(x_tot, y_tot)

Si on veut normaliser les données

In [21]:
from sklearn import preprocessing

In [92]:
x_scaled = preprocessing.scale(x_tot)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


Régression linéaire avec input normalisés

In [40]:
lr=linreg.fit(x_scaled, y_tot)

Tracés

In [66]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**Modèle de type MLP**

In [128]:
y_tot=log(y_tot)

On sépare les données entre apprentissage et test (60%)

In [129]:
from sklearn.model_selection import train_test_split

In [130]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_tot, test_size=0.6)

In [131]:
from sklearn.neural_network import MLPRegressor

In [132]:
nnet=MLPRegressor(hidden_layer_sizes=(11,),max_iter=5000,activation='tanh',alpha=0.05,solver='lbfgs')

In [134]:
y_train=y_train.reshape(4096,)

In [136]:
y_test=y_test.reshape(6144,)

In [137]:
nnet.fit(x_train,y_train)

MLPRegressor(activation='tanh', alpha=0.05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(11,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=5000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [148]:
nnet.score(x_test,y_test)

0.85254292201055981

**Exercice 1** : Tester avec d'autres paramètres

**Exercice 2** : Tester d'autres métamodèles

In [149]:
file_name="basesvm_avions.csv"

In [159]:
xy_classif=pd.read_csv(file_name,delimiter=';', header=None)

In [160]:
xy_classif.shape

(30000, 226)

In [305]:
from sklearn import svm, metrics

In [347]:
x_tot=np.array(xy_classif[np.arange(225)])

In [348]:
x_tot.shape

(30000, 225)

In [349]:
y_tot=np.array(xy_classif[225])

In [353]:
x0_train=x_tot[0:500,:]
x1_train=x_tot[10000:10500,:]
x2_train=x_tot[20000:20500,:]
x_train=np.concatenate((x0_train,x1_train,x2_train),axis=0)

In [355]:
y0_train=y_tot[0:500]
y1_train=y_tot[10000:10500]
y2_train=y_tot[20000:20500]
y_train=np.concatenate((y0_train,y1_train,y2_train),axis=0)

In [356]:
x0_test=x_tot[501:1500,:]
x1_test=x_tot[10501:11500,:]
x2_test=x_tot[20501:21500,:]
x_test=np.concatenate((x0_test,x1_test,x2_test),axis=0)

In [357]:
y0_test=y_tot[501:1500]
y1_test=y_tot[10501:11500]
y2_test=y_tot[20501:21500]
y_test=np.concatenate((y0_test,y1_test,y2_test),axis=0)

In [359]:
svm_iris=svm.SVC(kernel='rbf',C=10)

In [361]:
svm_iris.fit(x_train,y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [362]:
y_pred=svm_iris.predict(x_test)